## Imports

In [1]:
import torch
from torch import optim
from ignite.handlers import Checkpoint

In [2]:
%run -n train_classification.py

In [3]:
%run models/classification/__init__.py

In [4]:
%run datasets/__init__.py

In [5]:
%run models/checkpoint/__init__.py

In [7]:
!echo $CUDA_VISIBLE_DEVICES

0,1


In [8]:
DEVICE = torch.device('cuda', 1)
DEVICE

device(type='cuda', index=1)

## Load stuff

### Load datasets

In [6]:
dataset_name = 'cxr14'
kwargs = {
    'max_samples': 100,
    'batch_size': 30,
}

train_dataloader = prepare_data_classification(dataset_name, 'train', **kwargs)
val_dataloader = prepare_data_classification(dataset_name, 'val', **kwargs)
train_dataloader.dataset.size()

Loading train dataset...
Loading val dataset...


(100, 14)

### Load pretrained model

In [16]:
run_name = '0704_005511_covid-kaggle_tfs-small_lr1e-06'
compiled_model = load_compiled_model_classification(run_name, debug=True, device=DEVICE)

In [17]:
compiled_model.model

TransfusionCBRCNN(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(7, 7), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(7, 7), stride=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(7, 7), stride=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(7, 7), stride=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
  )
  (global_pool): Sequential(
    (0): 

### Create new model

In [6]:
lr = 0.000001
# run_name = f'cxr14_lr{lr}'
run_name = 'debugging'

model = init_empty_model('tfs-small',
                         train_dataloader.dataset.labels,
                         multilabel=train_dataloader.dataset.multilabel,
                         # pretrained_cnn=pretrained_cnn.base_cnn,
                         imagenet=True,
                         freeze=False,
                        ).to(DEVICE)

optimizer = optim.Adam(model.parameters(), lr=lr)

compiled_model = CompiledModel(model, optimizer)

## Train

In [8]:
%%time

train_model(run_name, compiled_model, train_dataloader, val_dataloader, n_epochs=1,
            loss_name='wbce',
            # print_metrics=['loss', 'acc', 'spec_covid', 'recall_covid'],
            print_metrics=['loss', 'acc', 'hamming'],
            device=DEVICE,
           )

--------------------------------------------------
Training...
Finished epoch 1/1 loss 594.5271 584.7553, acc 0.1898 0.1874, hamming 0.5279 0.4950, 0h 0m 5s
Average time per epoch:  0h 0m 5s
--------------------------------------------------
CPU times: user 28.7 s, sys: 2.23 s, total: 31 s
Wall time: 5.16 s


In [17]:
train_dataloader.dataset.__class__

mrg.datasets.cxr14.CXR14Dataset

In [20]:
compiled_model.model.base_cnn

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Debug stuff

### Test metrics

In [95]:
from ignite.metrics import Accuracy, Recall, Precision

In [143]:
%run ./metrics/classification/__init__.py
%run ./metrics/classification/specificity.py
%run ./metrics/classification/accuracy.py
%run ./metrics/classification/hamming.py

In [144]:
acc = MultilabelAccuracy(output_transform=_transform_remove_loss_and_round)
ham = Hamming(output_transform=_transform_remove_loss_and_round)

In [153]:
outputs = torch.tensor([[0, 1, 1],
                        [0.3, 0.7, 0.8],
                       ])
target = torch.tensor([[0, 0, 1],
                       [0, 1, 1],
                      ])

In [154]:
acc.reset()
acc.update(_transform_remove_loss_and_round((0, outputs, target)))
acc.compute()

0.6666666666666666

In [155]:
ham.reset()
ham.update(_transform_remove_loss_and_round((0, outputs, target)))
ham.compute()

0.16666666666666666

In [ ]:
sp = Specificity()
rec = Recall()
prec = Precision()

In [ ]:
fn = _get_transform_one_class(0)

In [ ]:
# outputs = torch.tensor([[1, 2, 1, 0, 0]])
# target = torch.tensor([[1, 0, 1, 1, 2]])
outputs = torch.tensor([[0, 20, -1],
                        [-40, 2, 3],
                        [17, 5, 6],
                       ])
target = torch.tensor([0, 0, 2])
outputs, target = fn((0, outputs, target))
outputs, target

In [ ]:
sp.reset()
sp.update((outputs, target))
sp.compute()

In [ ]:
rec.reset()
rec.update((outputs, target))
rec.compute().item()

In [ ]:
prec.reset()
prec.update((outputs, target))
prec.compute().item()

#### Test BCE loss

In [19]:
import torch
import numpy as np
from torch.nn.functional import binary_cross_entropy

In [41]:
%run losses/bce.py

In [45]:
EPS = 1e-5

In [86]:
target = torch.tensor([[1, 0, 0, 0, 0, 0],
                       [0, 0, 1, 0, 1, 0],
                       [0, 0, 0, 0, 0, 0],
                      ])
bce = WeigthedBCELoss()

In [90]:
output_o = torch.tensor([[1, 0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 1, 0],
                         [1, 1, 1, 1, 1, 1],
                        ]).float()
bce(output_o, target)

tensor(82.8836)

In [96]:
output_o = torch.tensor([[0, 0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 1, 0],
                         [1, 1, 1, 1, 1, 1],
                        ]).float()
bce(output_o, target)

tensor(151.9611)

In [97]:
output_o = torch.tensor([[1, 0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 1, 0],
                         [0, 0, 0, 0, 0, 0],
                        ]).float()
bce(output_o, target)

tensor(0.0004)

In [98]:
output_o = torch.tensor([[0, 0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 1, 0],
                         [0, 0, 0, 0, 0, 0],
                        ]).float()
bce(output_o, target)

tensor(69.0779)

In [110]:
output = output_o.clamp(min=EPS, max=1-EPS)
output

tensor([[1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 9.9999e-01, 1.0000e-05, 9.9999e-01, 1.0000e-05],
        [1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05, 1.0000e-05]])

In [94]:
total = np.prod(target.size())
positive = (target == 1).sum().item()
negative = total - positive
total, positive, negative

(18, 3, 15)

In [95]:
BP = total / positive
BN = total / negative
BP, BN

(6.0, 1.2)

In [85]:
target.size(), output.size()

(torch.Size([2, 3]), torch.Size([3, 6]))

In [116]:
left = (target * torch.log(output))
left

tensor([[-1.1513e+01, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
         -0.0000e+00],
        [-0.0000e+00, -0.0000e+00, -1.0014e-05, -0.0000e+00, -1.0014e-05,
         -0.0000e+00],
        [-0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00,
         -0.0000e+00]])

In [117]:
right = (1-target) * torch.log(1-output)
right

tensor([[-0.0000e+00, -1.0014e-05, -1.0014e-05, -1.0014e-05, -1.0014e-05,
         -1.0014e-05],
        [-1.0014e-05, -1.0014e-05, -0.0000e+00, -1.0014e-05, -0.0000e+00,
         -1.0014e-05],
        [-1.0014e-05, -1.0014e-05, -1.0014e-05, -1.0014e-05, -1.0014e-05,
         -1.0014e-05]])

In [118]:
-(weights*(left + right)).sum()

tensor(69.0779)

In [99]:
weights = torch.zeros(target.size())
weights

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [109]:
weights[target == 0] = BN
weights[target == 1] = BP
weights

tensor([[6.0000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000],
        [1.2000, 1.2000, 6.0000, 1.2000, 6.0000, 1.2000],
        [1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000]])

In [119]:
binary_cross_entropy(output_o, target.float(), weight=weights, reduction='sum')

tensor(600.)

In [120]:
bce(output_o, target)

tensor(69.0779)

In [122]:
def calc_conv_output_size(input_size, padding, kernel_size, stride, dilation=0):
    value = (input_size + 2*padding - dilation * (kernel_size - 1) - 1)
    value /= (stride)
    value += 1
    return value

In [124]:
conv = nn.Conv2d(3, 32, (8, 8), stride=4)